In [ ]:
import sys
import os
# Add project root to sys.path so src/ is importable
sys.path.insert(0, os.path.abspath('..'))

In [ ]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from src.model import LSTMAutoencoder
from src.preprocessing import load_data, normalize_data, segment_windows
from src.config import MODEL_PATH

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def main():
    # Load and preprocess data
    df = load_data()
    
    if 'label' in df.columns:
        df = df[df['label'] == 0]
        df = df.drop(columns=['label'])
    data_norm = normalize_data(df)
    
    # Use same window size as preprocessing
    windows = segment_windows(data_norm, window_size=50, step_size=25)
    
    # Check for empty windows
    if windows.shape[0] == 0:
        print("Not enough data for the chosen window size. Please decrease window_size or add more data.")
        return

    X = torch.tensor(windows, dtype=torch.float32)

    seq_len = X.shape[1]
    n_features = X.shape[2]
    embedding_dim = 64

    model = LSTMAutoencoder(seq_len, n_features, embedding_dim).to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    criterion = torch.nn.MSELoss()

    n_epochs = 30
    batch_size = 32
    train_loader = DataLoader(TensorDataset(X), batch_size=batch_size, shuffle=True)

    model.train()
    for epoch in range(n_epochs):
        epoch_loss = 0
        for batch in train_loader:
            batch_x = batch[0].to(DEVICE)
            output = model(batch_x)
            loss = criterion(output, batch_x)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f"Epoch {epoch+1}/{n_epochs}, Loss: {epoch_loss/len(train_loader):.4f}")

    MODEL_PATH.parent.mkdir(parents=True, exist_ok=True)
    torch.save(model.state_dict(), str(MODEL_PATH))
    print(f"Model saved to {MODEL_PATH}")

if __name__ == "__main__":
    main()

In [ ]:
import matplotlib.pyplot as plt

losses = [0.2510, 0.1741, 0.0924, 0.0676, 0.0544, 0.0425, 0.0376, 0.0313, 0.0241,
          0.0196, 0.0170, 0.0147, 0.0123, 0.0144, 0.0122, 0.0098, 0.0105, 0.0086,
          0.0078, 0.0079, 0.0070, 0.0068, 0.0059, 0.0063, 0.0064, 0.0069, 0.0062,
          0.0056, 0.0070, 0.0103]

plt.plot(range(1, 31), losses, marker='o')
plt.title('Training Loss per Epoch')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.grid(True)
plt.show()